<a href="https://colab.research.google.com/github/rafzieli/Garmin_dashboard/blob/main/colab_dash.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Setup

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/garmin_colab

/content/drive/MyDrive/garmin_colab


In [3]:
! pip install jupyter_dash

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 9.8 MB 8.6 MB/s 
     |████████████████████████████████| 357 kB 31.3 MB/s 
  Created wheel for retrying: filename=retrying-1.3.3-py3-none-any.whl size=11447 sha256=10deee038b6ed0d03a24ab4e0b0430cc37a70b2e1394673fd66e35ac8c3f2715
  Stored in directory: /root/.cache/pip/wheels/f9/8d/8d/f6af3f7f9eea3553bc2fe6d53e4b287dad18b06a861ac56ddf
Successfully built retrying


In [4]:
! pip install dash


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
! pip install dash_daq

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 642 kB 7.6 MB/s 
  Created wheel for dash-daq: filename=dash_daq-0.5.0-py3-none-any.whl size=669714 sha256=8e52cc455759da30d1f6a1f6262701cd480082e5202a34409644d09a879cd865
  Stored in directory: /root/.cache/pip/wheels/fe/54/53/a8d448db5592874db4313240571ca2c069e55f6a6b29bf5847
Successfully built dash-daq


In [16]:
!python pandas_data_wrangling.py

# Dash

In [22]:
import dash
from dash import dcc
from dash import html
import plotly.graph_objects as go
from dash.dependencies import Input, Output
from jupyter_dash import JupyterDash
from dash import dash_table
from dash.dash_table import FormatTemplate
import pandas as pd
import numpy as np
from datetime import datetime as dt
import pandas_data_wrangling
import dash_daq as daq

df = pd.read_csv('./dashboard_df.csv')
numeric_cols = ['Distance', 'Calories', 'AvgHR', 'MaxHR', 'AvgRunCadence', 'MaxRunCadence', 'NumberofLaps',
                'TotalAscent',
                'TotalDescent', 'MinElevation', 'MaxElevation']
timedelta_cols = ['Time', 'BestLapTime', 'MovingTime', 'ElapsedTime']

# converting dtypes
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric, errors='coerce', axis=1)
df[timedelta_cols] = df[timedelta_cols].apply(pd.to_timedelta, errors='coerce', axis=1)


# df.Date = [date[:10] for date in list(df.Date)]
# df.Date = df.Date.apply(pd.to_datetime)

def format_timedelta(td):
    minutes, seconds = divmod(td.seconds + td.days * 86400, 60)
    hours, minutes = divmod(minutes, 60)
    return '{:02d}:{:02d}'.format(minutes, seconds)


# as watches not always deliver us information about avgSpeed/avgPace, we need to calculate it in additional column
df['TimeInSec'] = [item.total_seconds() for item in list(df.Time)]
df['TimeInHour'] = round(df.TimeInSec / 3600, 2)
df['AvgSpeed'] = round((df.Distance / (df.TimeInSec)) * 3600, 1)
df['AvgPaceCountFloat'] = round((df.TimeInSec / df.Distance) / 60, 2)
df['AvgPaceCountTimedelta'] = df.Time / df.Distance
df['AvgPaceCountTimedelta'] = pd.to_timedelta(df['AvgPaceCountTimedelta'])
df['AvgPaceCountString'] = df.apply(lambda x: format_timedelta(x['AvgPaceCountTimedelta']), axis=1)

external_stylesheets = ['https://codepen.io/chriddyp/pen/bWLwgP.css']


theme = {
    'dark': True,
    'detail': '#007439',
    'primary': '#00EA64',
    'secondary': '#6E6E6E',
}


app = JupyterDash(__name__, external_stylesheets=external_stylesheets)

app.layout = html.Div([

    html.H1('Garmin Dashboard',
            style={
                'font-fmaily': 'Helvetica',
                'text-align': 'center'
            }),

    html.Hr(),

    # 1st line dropdowns
    html.Div([
        # Athletes div
        html.Div([
            html.H5('Athletes:'),
            html.Div(
                dcc.Dropdown(id='athlete-drop',
                             options=df['Athlete'].unique(),
                             multi=True,
                             searchable=True,
                             placeholder='Select Athlete to compare sport data',
                             value=df.Athlete.unique()[0]),
            )
        ],
            style={'width': '40%',
                   'margin-right': '5%',
                   'margin-left': '5%'}),
        # Sport div - moze dac radio?
        html.Div([
            html.H5('Activity Type:'),
            html.Div(
                dcc.RadioItems(id='activity-radio',
                               options=sorted(df.ActivityType.unique()),
                               value='Cycling'
                               )
            )
        ],
            style={'width': '40%',
                   'margin-right': '5%',
                   'margin-left': '5%'})
    ], style={'display': 'flex'}),

    # 2nd line dropdowns
    html.Div([
        # Property div
        html.Div([
            html.H5('Properties:'),
            html.Div(
                dcc.Dropdown(id='property-drop',
                             options=[
                                 {'label': 'Total distance', 'value': 'total_dist'},
                                 {'label': 'Average distance per activity', 'value': 'avg_dist_act'},
                                 {'label': 'Average speed per activity', 'value': 'avg_speed_act'},
                                 {'label': 'Average pace per activity', 'value': 'avg_pace_act'},
                                 {'label': 'Total time', 'value': 'total_time'},
                                 {'label': 'Total calories burned', 'value': 'total_calories'}
                             ],
                             value='total_dist',
                             # multi=True,
                             searchable=True,
                             placeholder='Select property to compare'),
            )
        ],
            style={'width': '40%',
                   'margin-right': '5%',
                   'margin-left': '5%'}),
        # Date div
        html.Div([
            html.H5('Time period:'),
            html.Div(
                dcc.DatePickerRange(id='date-picker',
                                    start_date=df.Date.min(),
                                    end_date=df.Date.max(),
                                    min_date_allowed=df.Date.min(),
                                    max_date_allowed=df.Date.max(),
                                    display_format='YYYY-MM-DD'
                                    )
            )
        ],
            style={'width': '40%',
                   'margin-right': '5%',
                   'margin-left': '5%'})
    ], style={'display': 'flex'}),

    html.Hr(),
    html.Div([
        dcc.Graph(id='bar-graph'),
        ##dcc.Graph(id='scatter-graph')
    ]),

    html.Div(html.H3(id='highscores-title'), style={'text-align': 'center'}),
    html.Div([
        html.Div([
            html.Div(html.H5(id='gauge-1-title'), style={'text-align': 'center'}),
            daq.Gauge(
                id='gauge-1',
                label='Distance',
                showCurrentValue=True,
                units='km',
                color=theme['primary']), ],
            style={'width': '15%', 'margin-right': '2.5%', 'margin-left': '2.5%'}),
        html.Div([
            html.Div(html.H5(id='gauge-2-title'), style={'text-align': 'center'}),
            daq.Gauge(
                id='gauge-2',
                label='Time',
                showCurrentValue=True,
                units='h',
                color=theme['primary'])],
            style={'width': '15%', 'margin-right': '2.5%', 'margin-left': '2.5%'}),
        html.Div([
            html.Div(html.H5(id='gauge-3-title'), style={'text-align': 'center'}),
            daq.Gauge(
                id='gauge-3',
                label='Biggest ascent',
                showCurrentValue=True,
                units='m',
                color=theme['primary'])],
            style={'width': '15%', 'margin-right': '2.5%', 'margin-left': '2.5%'}),
        html.Div([
            html.Div(html.H5(id='gauge-4-title'), style={'text-align': 'center'}),
            daq.Gauge(
                id='gauge-4',
                label='Most calories burned',
                showCurrentValue=True,
                units='kcal',
                color=theme['primary'])],
            style={'width': '15%', 'margin-right': '2.5%', 'margin-left': '2.5%'}),
        html.Div([
            html.Div(html.H5(id='gauge-5-title'), style={'text-align': 'center'}),
            daq.Gauge(
                id='gauge-5',
                showCurrentValue=True,
                color=theme['primary'])],
            style={'width': '15%', 'margin-right': '2.5%', 'margin-left': '2.5%'})
    ], style={'display': 'flex'}),

])


@app.callback(
    [Output('bar-graph', 'figure'),
     Output('gauge-1-title', 'children'),
     Output('gauge-2-title', 'children'),
     Output('gauge-3-title', 'children'),
     Output('gauge-4-title', 'children'),
     Output('gauge-5-title', 'children')],
    [Input('athlete-drop', 'value'),
     Input('activity-radio', 'value'),
     Input('property-drop', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def render_graph_bar_update_gauge_names(athletes, activity, property, start_date, end_date):
    df_temp = df[(df.Athlete.isin(list(athletes))) &
                 (df.Date > start_date) &
                 (df.Date < end_date) &
                 (df.ActivityType == activity)]

    property_dict = {
        'total_dist': df_temp.groupby('Athlete').sum()['Distance'],
        'avg_dist_act': round(df_temp.groupby('Athlete').mean()['Distance'], 2),
        'avg_speed_act': round(df_temp.groupby('Athlete').mean()['AvgSpeed'], 2),
        'avg_pace_act': round(df_temp.groupby('Athlete').mean()['AvgPaceCountFloat'], 2),
        'total_time': round(df_temp.groupby('Athlete').sum()['TimeInHour'], 2),
        'total_calories': round(df_temp.groupby('Athlete').sum()['Calories'], 2)
    }
    
    x = df_temp.Athlete.unique()
    y = list(property_dict[str(property)].values)

    fig = go.Figure(
        data=[go.Bar(
            x=x,
            y=y,
            marker={'color': 'darkgreen'},
            text=y
        )],
        layout=go.Layout(
            title=go.layout.Title(text=f'{activity} activities from {start_date} to {end_date}'),
            xaxis_title='Athlete',
            yaxis_title=f'{property}',
            barmode='stack',
            xaxis={'categoryorder': 'total descending'},
            font={'size': 16,
                  'color': 'darkblue'},
            paper_bgcolor='darkgray',
            plot_bgcolor='lightgray'
        )
    )
    name1 = df_temp[df_temp['Distance'] == df_temp['Distance'].max()]['Athlete']
    name2 = df_temp[df_temp['TimeInHour'] == df_temp['TimeInHour'].max()]['Athlete']
    name3 = df_temp[df_temp['TotalAscent'] == df_temp['TotalAscent'].max()]['Athlete']
    name4 = df_temp[df_temp['Calories'] == df_temp['Calories'].max()]['Athlete']
    name5 = df_temp[df_temp['AvgSpeed'] == df_temp['AvgSpeed'].max()]['Athlete']

    return fig, name1, name2, name3, name4, name5


@app.callback(
    [Output('gauge-1', 'value'),
     Output('gauge-2', 'value'),
     Output('gauge-3', 'value'),
     Output('gauge-4', 'value'),
     Output('gauge-5', 'value')],
    [Input('athlete-drop', 'value'),
     Input('activity-radio', 'value'),
     Input('property-drop', 'value'),
     Input('date-picker', 'start_date'),
     Input('date-picker', 'end_date')]
)
def update_gauge_value(athletes, activity, property, start_date, end_date):
    #print(athletes, activity, property, start_date, end_date)

    df_temp = df[(df.Athlete.isin(list(athletes))) &
                 (df.Date > start_date) &
                 (df.Date < end_date) &
                 (df.ActivityType == activity)]

    value1 = round(df_temp['Distance'].max(), 1)
    print(f'value1 = {value1}')
    value2 = df_temp['TimeInHour'].max()
    value3 = df_temp['TotalAscent'].max()
    value4 = df_temp['Calories'].max()

    if activity == 'Cycling':
        value5 = round(df_temp['AvgSpeed'].max(), 1)
    else:
        value5 = round(df_temp['AvgPaceCountFloat'].min(), 1)
    print(f'value5 = {value5}')

    return value1, value2, value3, value4, value5


@app.callback(
    [Output('gauge-1', 'max'),
     Output('gauge-2', 'max'),
     Output('gauge-3', 'max'),
     Output('gauge-4', 'max'),
     Output('gauge-5', 'max'),
     Output('gauge-5', 'label'),
     Output('gauge-5', 'units')],
    [Input('activity-radio', 'value')]
)
def update_gauge_properties(activity):
    if activity == 'Cycling':
        max1 = 300
        max2 = 24
        max3 = 3000
        max4 = 6000
        max5 = 70
        label5 ='Highest Average Speed'
        units5 = 'km / h'
    else:
        max1 = 50
        max2 = 24
        max3 = 3000
        max4 = 6000
        max5 = 8
        label5 = 'Best Pace'
        units5 = 'min / km'
    return max1, max2, max3, max4, max5, label5, units5


@app.callback(
    Output('highscores-title', 'children'),
    Input('activity-radio', 'value')
)
def update_highscores_title(activity):
    return f'Single activity highscores - {activity}'


if __name__ == '__main__':
    app.run_server(mode='external', port = 8090, dev_tools_ui=True, #debug=True,
              dev_tools_hot_reload =True, threaded=True)


Dash app running on:


<IPython.core.display.Javascript object>

In [10]:
df


,Unnamed: 0,Athlete,ActivityType,Date,Title,Distance,Calories,Time,AvgHR,MaxHR,AvgRunCadence,MaxRunCadence,BestPace,TotalAscent,TotalDescent,BestLapTime,NumberofLaps,MovingTime,ElapsedTime,MinElevation,MaxElevation,TimeInSec,AvgSpeed,AvgPaceCountTimedelta,AvgPaceCountString,TimeInHour,AvgPaceCountFloat
0,0,Ola,Cycling,2022-05-15,Warszawa Kolarstwo,32.24,745.0,0 days 01:38:03,136.0,170.0,NaN,NaN,NaN,63.0,49.0,NaT,7.0,0 days 01:36:50,0 days 01:45:11,82.0,111.0,5883.0,19.7,0 days 00:03:02.475186104,03:02,1.63,3.04
1,1,Ola,Cycling,2022-05-08,Warszawa Kolarstwo,24.82,646.0,0 days 01:17:07,139.0,155.0,NaN,NaN,NaN,69.0,94.0,NaT,5.0,0 days 01:15:52,0 days 01:19:45,10.0,31.0,4627.0,19.3,0 days 00:03:06.422240128,03:06,1.29,3.11
2,2,Ola,Hiking,2022-03-06,Warszawa Chodzenie,5.35,276.0,0 days 01:02:21,99.0,122.0,109.0,128.0,NaN,23.0,4.0,NaT,7.0,0 days 00:58:51,0 days 01:02:21,98.0,118.0,3741.0,5.1,0 days 00:11:39.252336448,11:39,1.04,11.65
3,3,Ola,Running,2022-01-16,Sopot Bieganie,5.51,335.0,0 days 00:31:52,162.0,179.0,168.0,211.0,NaN,16.0,2.0,NaT,6.0,0 days 00:31:44,0 days 00:31:52,-1.0,16.0,1912.0,10.4,0 days 00:05:47.005444646,05:47,0.53,5.78
4,4,Ola,Cycling,2022-01-15,Gdańsk Kolarstwo,22.10,423.0,0 days 01:12:13,123.0,158.0,NaN,NaN,NaN,47.0,50.0,NaT,5.0,0 days 01:11:28,0 days 01:23:15,-25.0,1.0,4333.0,18.4,0 days 00:03:16.063348416,03:16,1.20,3.27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1774,1865,Zielak,Hiking,2019-01-13,Kolbudy Spacerowanie,4.00,257.0,0 days 00:53:50,100.0,130.0,93.0,215.0,NaN,65.0,52.0,NaT,5.0,0 days 00:50:30,0 days 00:53:50,100.0,229.0,3230.0,4.5,0 days 00:13:27.500000,13:27,0.90,13.46
1775,1866,Zielak,Hiking,2019-01-12,Gdańsk Spacerowanie,2.92,184.0,0 days 00:36:12,100.0,116.0,94.0,238.0,NaN,67.0,62.0,NaT,3.0,0 days 00:33:23,0 days 00:36:12,172.0,235.0,2172.0,4.8,0 days 00:12:23.835616438,12:23,0.60,12.40
1776,1867,Zielak,Running,2019-01-08,Gdańsk Biegi,2.57,190.0,0 days 00:15:12,152.0,183.0,149.0,237.0,NaN,29.0,27.0,NaT,3.0,0 days 00:15:03,0 days 00:15:12,43.0,74.0,912.0,10.1,0 days 00:05:54.863813229,05:54,0.25,5.91
1777,1868,Zielak,Running,2019-01-06,Gdańsk Biegi,2.23,151.0,0 days 00:11:36,153.0,187.0,153.0,166.0,NaN,27.0,29.0,NaT,3.0,0 days 00:11:27,0 days 00:11:36,-30.0,-5.0,696.0,11.5,0 days 00:05:12.107623318,05:12,0.19,5.20
